In [6]:
from dotenv import load_dotenv
import os
from langchain_community.tools.tavily_search import TavilySearchResults

load_dotenv()
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [39]:
search = TavilySearchResults()

search.invoke("How many calories on average does leg press burn if weight is 80kg")

[{'title': 'How much calories does a single rep of leg press burn? - Quora',
  'url': 'https://www.quora.com/How-much-calories-does-a-single-rep-of-leg-press-burn',
  'content': 'It has to be done for health and if you want to have weight control. ... Calories burned per hour at 80kg weight: Rowing - 500–1000. Cycling',
  'score': 0.8051581},
 {'title': 'Calories Burned for Squats on Leg Press Machine - SparkPeople',
  'url': 'https://www.sparkpeople.com/resource/calories_burned.asp?exercise=226&type=1',
  'content': "Calories Burned - Squats on Leg Press Machine\n\nFind out how many calories you burn for Squats on Leg Press Machine. The number of calories you burn while exercising is dependent on the exercise you do, your weight, and the time spent doing the exercise. Use the calories burned calculator below to see how many calories you burned during your workout.\r\n\t\t\t\t\t\t\r\n\t\n\n\nYour Weight in Pounds:\r\n\t\t\t\t\t\t\n\n\n\nMinutes Spent Exercising:\r\n\t\t\t\t\t\t\n\n\n\n

In [17]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load web content
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
print(docs)

# Split the content
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

# Use Sentence Transformers model (lightweight & fast)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create vector store
vector = FAISS.from_documents(documents, embedding_model)

# Get retriever
retriever = vector.as_retriever()


In [15]:
retriever.invoke("is langsmith open source?")

[Document(id='85eeec1d-705b-43c6-b75a-ecabc98a8031', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(id='5abe06a0-aa85-4891-a843-0c6205cb3b59', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication metho

In [19]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [20]:
tools = [search, retriever_tool]

In [31]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

llm = ChatOllama(model='llama3.2')

response = llm.invoke([
    HumanMessage(content="Tell me a fun fact about space, make it short and sweet")
])

response.content

"Did you know that there is a giant storm on Jupiter that has been raging for at least 187 years? The Great Red Spot is a swirling anticyclonic storm that's larger than Earth itself!"

In [32]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [33]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm,tools,prompt)

In [34]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
agent_executor.invoke({"input": "hi there!"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'hi there!',
 'output': "To get started with LangSmith, you can begin by adding tracing to your application. This will allow you to closely monitor and evaluate your LLM application. Additionally, you can create dashboards to view key metrics such as RPS, error rates, and costs. \n\nFor more information on getting started with LangSmith, I recommend checking out the [LangSmith documentation](https://langsmith.io/docs) or reaching out to their support team for assistance.\n\nIf you're looking for help with a specific aspect of LangSmith, feel free to ask and I'll do my best to assist you."}